In [ ]:
import torch
import numpy as np
import pandas as pd

# My libraries
import sys
sys.path.append('../')
import utils

## General parameters

In [ ]:
numberOfTimeStep = 14
folders = ["s1", "s2", "s3"]

norm = "robustNorm"
build_grapb_by = "dtw-gower"

device = torch.device('cuda:1')
print("Seleccionando la segunda GPU:", torch.cuda.get_device_name(device))

## Train phase

In [ ]:
# Always the same values, do not touch. 
in_dim_GCN = 1
out_dim_GCN = 1

# number of max epochs 
n_epochs = 1000

# Early stopping configuration
early_stopping_patience = 50

# Hyperparameters to be optimized (change this values)
h_dropout = [0.0, 0.15, 0.3]
h_learning_rate = [1e-3, 1e-2, 5e-2, 0.1]
h_decay = [0, 1e-5, 1e-4, 1e-3, 1e-2]
h_hid_lay = [4, 8, 16, 32, 64]
h_layers = [1, 2, 3, 4, 5, 6]
seed = [42, 76, 124, 163, 192, 205, 221, 245, 293]

fc_layer = [[80, out_dim_GCN]]

# Parameters to define type of GCNN and type of output.
typeGCN = "standard_gcnn"
K = [0]

params = {# Hyperparameters
         'h_layers':h_layers, 'n_epochs':n_epochs, 
          'h_dropout': h_dropout, 'h_learning_rate': h_learning_rate, 
          'h_decay':h_decay, 'h_hid_lay': h_hid_lay, 'K':K,
          'fc_layer': fc_layer,
          # seed to set initialization hyperparameters
          'seed': seed, 
          # Type of output GCN
          'typeGCN': typeGCN,
          # Dimensions of GCN (input/output)
          'in_dim_GCN': in_dim_GCN, 'out_dim_GCN': out_dim_GCN,
          # Patiente
          'early_stopping_patience':early_stopping_patience}

In [ ]:
way_to_build_graph = "smoothness"
numberOfFeatures = 80

In [ ]:
import time
import json

best_result_by_split = {}
train_times_by_split = {}

for carp in range(len(folders)):
    torch.cuda.empty_cache()

    # Load data for the current split
    X_train_vec, X_val_vec, X_test_vec, y_train, y_val, y_test = utils.load_data(norm, device, folders[carp], SG=True)
    A = pd.read_csv("../../step2_graphRepresentation/" + way_to_build_graph + "/" + folders[carp] + "/ProdGraph_Xtr_" + norm + "_th_0.95.csv")
    A = A.iloc[0:numberOfFeatures, 0:numberOfFeatures]
    A = torch.tensor(np.array(A), dtype=torch.float32)

    print(f"\n===========> TRAIN-VAL PHASE for folder {folders[carp]} ==================")

    # Measure training time
    start_time = time.time()
    bestHyperparameters = utils.train_val_phase(A, X_train_vec, X_val_vec, y_train, y_val, params, device)
    end_time = time.time()
    elapsed_time = round(end_time - start_time, 2)

    print(f"<========== END TRAIN-VAL PHASE for folder {folders[carp]} | Elapsed time: {elapsed_time:.2f} seconds ==============\n")

    # Store best hyperparameters
    best_result_by_split[folders[carp]] = {
        "best_hyperparameters": bestHyperparameters
    }

# Save best hyperparameters
path_hyper = "../hyperparameters/" + way_to_build_graph + "/#E1-SingleGraph_th_0.95.json"
utils.saveBestHyperparameters(best_result_by_split, path_hyper)

torch.cuda.empty_cache()

## Validation phase

In [ ]:
import time
import json
import numpy as np
import torch

folders = ["s1", "s2", "s3"]
best_result_by_split = utils.loadBestHyperparameters(
    "../hyperparameters/" + way_to_build_graph + "/#E1-SingleGraph_th_0.95.json"
)

typeOfGraph = "ProdGraph"
path_A = typeOfGraph + "_Xtr_" + norm + "_th_0.975.csv"  # File may still use 0.975 as suffix

# Dictionary to store inference times
inference_times_by_split = {}

# Containers for results
results = {}
importance_nodes = {}
fc_classifiers = {}
gnn_models = {}

# Inference loop per split
for folder in folders:

    start_time = time.time()
    res, imp_nodes, fc_cls, gnn_mod = utils.val_model(
        {folder: best_result_by_split[folder]},
        typeOfGraph, params, [folder], norm, device, path_A, way_to_build_graph, SG=True
    )
    end_time = time.time()
    elapsed = round(end_time - start_time, 2)

    inference_times_by_split[folder] = {
        "inference_time_seconds": elapsed
    }

    # Collect results
    for k in res:
        if k not in results:
            results[k] = []
        results[k].append(res[k][0])
    importance_nodes[folder] = imp_nodes
    fc_classifiers[folder] = fc_cls
    gnn_models[folder] = gnn_mod

# Print results per split
keys = list(results.keys())
for c in range(len(folders)):
    print("================= SPLIT " + str(folders[c]) + " ===================")
    for k in keys:
        print(k + ": " + str(np.round(results[k][c] * 100, 2)))

# Print average and standard deviation
print()
str_result = ""
for k in keys:
    avg = np.mean(results[k])
    std = np.std(results[k])
    print(f"{k}: {np.round(avg * 100, 2)} +- {np.round(std * 100, 2)}")

    if k in ["test_acc", "roc_auc", "auc_pr"]:
        str_result += f"{np.round(avg * 100, 2)} +- {np.round(std * 100, 2)} & "
    elif k == "specificity":
        str_result += f"{np.round(avg * 100, 2)} +- {np.round(std * 100, 2)} \\\ "

print("\n", str_result)

# Save inference times to JSON file
path_times = "../hyperparameters/" + way_to_build_graph + "/#E1-InferenceTimes_th_0.95.json"
with open(path_times, 'w') as f:
    json.dump(inference_times_by_split, f, indent=4)

print(f"\nInference times saved to: {path_times}")

torch.cuda.empty_cache()


In [ ]:
72.7 +- 1.87 & 63.52 +- 2.35 & 70.15 +- 1.11 & 42.37 +- 2.21 \\ 